In [ ]:
import tensorflow as tf
import numpy as np


datapoint_size = 1000
batch_size = 1
steps = 100
actual_W = 2
actual_b = 10
learn_rate = 0.001
log_file = "/tmp/feature_1_batch_1"

# Model linear regression y = Wx + b
x = tf.placeholder(tf.float32, [None, 1], name="x")
W = tf.Variable(tf.zeros([1,1]), name="W")
b = tf.Variable(tf.zeros([1]), name="b")
with tf.name_scope("Wx_b") as scope:
  product = tf.matmul(x,W)
  y = product + b

# Add summary ops to collect data
W_hist = tf.histogram_summary("weights", W)
b_hist = tf.histogram_summary("biases", b)
y_hist = tf.histogram_summary("y", y)

y_ = tf.placeholder(tf.float32, [None, 1], name="y_")

# Cost function sum((y_-y)**2)
with tf.name_scope("cost") as scope:
  cost = tf.reduce_mean(tf.square(y_-y))
  cost_sum = tf.scalar_summary("cost", cost)

# Training using Gradient Descent to minimize cost
with tf.name_scope("train") as scope:
  train_step = tf.train.GradientDescentOptimizer(learn_rate).minimize(cost)

all_xs = []
all_ys = []
for i in range(datapoint_size):
  # Create fake data for y = W.x + b where W = 2, b = actual_b
  all_xs.append(i%10)
  all_ys.append(actual_W*(i%10)+actual_b)

all_xs = np.transpose([all_xs])
all_ys = np.transpose([all_ys])

sess = tf.Session()

# Merge all the summaries and write them out to /tmp/mnist_logs
merged = tf.merge_all_summaries()
writer = tf.train.SummaryWriter(log_file, sess.graph_def)

init = tf.initialize_all_variables()
sess.run(init)

for i in range(steps):
  if datapoint_size == batch_size:
    batch_start_idx = 0
  elif datapoint_size < batch_size:
    raise ValueError("datapoint_size: %d, must be greater than batch_size: %d" % (datapoint_size, batch_size))
  else:
    batch_start_idx = (i * batch_size) % (datapoint_size - batch_size)
  batch_end_idx = batch_start_idx + batch_size
  batch_xs = all_xs[batch_start_idx:batch_end_idx]
  batch_ys = all_ys[batch_start_idx:batch_end_idx]
  xs = np.array(batch_xs)
  ys = np.array(batch_ys)
  # Record summary data, and the accuracy every 10 steps
  if i % 10 == 0:
    all_feed = { x: all_xs, y_: all_ys }
    result = sess.run(merged, feed_dict=all_feed)
    writer.add_summary(result, i)
  else:
    feed = { x: xs, y_: ys }
    sess.run(train_step, feed_dict=feed)
    print("y: %s" % sess.run(y, feed_dict=feed))
    print("y_: %s" % ys)
    print("cost: %f" % sess.run(cost, feed_dict=feed))
  print("After %d iteration:" % i)
  print("W: %f" % sess.run(W))
  print("b: %f" % sess.run(b))

Basic Operation
Tensor Types
Project Speed Dating
Placeholders and Feeding inputs
Lazy Loading

Provides extensive suite of functions and classes to build models from scratch. 

Off the self models aren't focus of tensoflow.

Data Flow Grpahs:

It separates computation from actutal execution. Define compute as graph, assemble graphs and set sessions to execute them.

Tensor: A n-dimensional matrix. In tesnorflow the information flows between nodes as tensors and compute applies to 
    tensors for tensor transformation.
    
    0-Dimensional --> Scalar
    
    1-Dimensional --> Vector
    
    2-Dimensional --> Matrix
    
    > 2 Dimensional --> Tensors

Nodes: Variables, constants and operators
Edges: Tensors

In [2]:
import tensorflow as tf
a = tf.add(2,3)

In [ ]:
print(a)

We have just defined a computational graph a where 2 and 3 are nodes pass on to an add operation to create another node a.
But a is still not executed to generate addition of 2 & 3 to 5 because graph hasn't been executed yet.

Seeing the output a --> an "add" operation with output of tensor shape 0 (scalar) of int32 data type.

In [ ]:
b = tf.add(3.0, 5.0)
print(b)

Now, to compute actual output of the add operation defined by the Graph we need to create a session. The objective of having 
session is to utilize all the compute resources available (cpu or gpu) and parallelize operation. 
This is concept of lazy execution.

In [ ]:
sess = tf.Session()
sess.run(a)
sess.close()

What happened with sess.run command is that tensorflow saw node named "a" and execute whatever necessary to compute node a.

In [ ]:
with tf.Session() as sess: # This will do a implicit close of session once it come out of with
    print(sess.run(a))

In [ ]:
#lets look at another example
x = 2
y = 3

op1 = tf.add(x,y)
op2 = tf.mul(x,y)
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
    print(sess.run(op3))

In [ ]:
#lets look at another example
x = 2
y = 3

op1 = tf.add(x,y)
op2 = tf.mul(x,y)
useless = tf.mul(x, op1)
op3 = tf.pow(op2, op1)

with tf.Session() as sess:
    op3, not_useless = sess.run([op3, useless])
print(op3)
print(not_useless)

Tesnsorflow allows breaking graphs into many sub-graphs to execute them into mutiple CPU or GPUs.

In [ ]:
#lets look at same example another way
x = 2
y = 3

op1 = tf.add(x,y)
op2 = tf.mul(x,y)
useless = tf.mul(x, op1)
op3 = tf.pow(op2, op1)

fetches = [op3, useless]
with tf.Session() as sess:
    op3, not_useless = sess.run(fetches, feed_dict=None, options = None, run_metadata=None)
print(op3)
print(not_useless)

In [ ]:
#to put a specific part of the graph into specific cpu or gpu


#Create a grpah
with tf.device("/gpu:2"):
    a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], shape = [1, 5], name = 'a')
    b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], shape = [5, 1], name = 'b')
    #b = tf.transpose(b)
    c = tf.matmul(a, b)
    #create a session with log_device_placement true
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    
    #Run this session
    print(sess.run(c))

In [ ]:
sess.close()

In [3]:
a = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], shape = [1, 5], name = 'a')
b = tf.constant([1.0, 2.0, 3.0, 4.0, 5.0], shape = [5, 1], name = 'b')
    #b = tf.transpose(b)
c = tf.matmul(a, b)
    #create a session with log_device_placement true
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess = tf.Session()    
    #Run this session
print(sess.run(c))

[[ 55.]]


Multiple graphs needs multiple sessions, each will try to use all the available resources by default.

Can't pass data between graphs without using python or numpy which isn't distributed.

Its better to have disconnected subgraphs within a graph. 

In [12]:
#create a graph - other than default graphs


g = tf.Graph()
with g.as_default():
    x = tf.add(2, 3)
    sess = tf.Session(graph=g)
    with tf.Session() as sess:
        print(sess.run(x))

5


In [13]:
g = tf.Graph()
with g.as_default():
    a = 3
    b = 2
    x = tf.add(a, b)
sess = tf.Session(graph=g)
sess.run(x)

5

In [14]:
#to get the handle of the default graph
g = tf.get_default_graph()

In [ ]:
#do Not mix graphs
g1 = tf.get_default_graph()
g2 = tf.Graph()

with g1.as_default():
    a = tf.constant(3)
with g2.as_default():
    b = tf.constant(2)

Why Graphs:
    Only run part of the graph that you need to execute - saves computation
    Break graphs into small, differential pieces to facilitate auto differentiation
    Facilitate distributed computation into multiple CPUs or GPUs.
    Many machine learning models are created as directed graph already.